In [1]:
from typing import Iterable
from limes_x.solver import Application, Endpoint, Transform, Namespace, Solve

NS = Namespace()
_ids = 0
_ids_map = {}
def make_node(name: str, ins: Iterable[str], outs:Iterable[str]) -> Transform:
    tr = Transform(NS)
    for x in ins:
        tr.AddRequirement([x])
    for x in outs:
        tr.AddProduct([x])
    return tr

def _test(name, transforms: list[Transform], given: set[str], targets:set[str], expected: list):
    print(f"{name}:")

    parsed_given: list[Endpoint] = []
    parsed_target = Transform(NS)
    for x in targets:
        parsed_target.AddRequirement(x)
    for x in given:
        parsed_given.append(Endpoint(NS, {x}))

    result = Solve(parsed_given, parsed_target, transforms)
    # if result is None or result == False: result = []

    def _print_sol(x: list[Application]):
        for a in x:
            print(a)

    failed = False
    if len(result.solution[:-1]) != len(expected):
        print(f"sol len diff")
        print(expected)
        failed = True

    have = given
    for appl in result.solution:
        used = {e for g in [e.properties for e in appl.used] for e in g}
        if not used.issubset(have):
            print(f"transform missing requirements {appl}")
            failed = True
            break

        have |= {e for g in [e.properties for e in appl.produced] for e in g}

    if not targets.issubset(have):
        print(f"failed to produce {targets - have}")
        failed = True
    
    if not failed: print(f"{name} solved in {result.steps} steps")
    else:
        print(result.message)
        _print_sol(result.solution)
    print()
    return result

In [2]:
r = _test(
    'complex',[
        make_node('sa', {'s'}, {'a', '_a'}),

        make_node('ab', {'a'}, {'b'}),
        make_node('ay', {'a'}, {'y'}),
        make_node('bc', {'b'}, {'c'}),
        make_node('ca', {'c'}, {'a'}),
        make_node('cx', {'c'}, {'x'}),
        make_node('jx', {'j'}, {'x'}),

        make_node('_ab', {'_a'}, {'_b'}),
        make_node('_ay', {'_a'}, {'_y'}),
        make_node('_ac', {'_a'}, {'_c'}),
        make_node('_bc.x', {'_b', '_c'}, {'_x'}),
        make_node('_jx', {'_j'}, {'_x'}),

        make_node('+1', {'b', '_b'}, {'+1'}),
        make_node('+2', {'c', '_b', 'a'}, {'+2', 'a'}),
        make_node('+3', {'+1', '+2', '_y', '_c', 'c'}, {'j', '+3', 'x'}),

        make_node('end', {'+1', '+2', '+3'}, {'+x'})
    ],
    {'s'}, {'+x'},
    [
        make_node('sa', {'s'}, {'a', '_a'}),
        make_node('ab', {'a'}, {'b'}),
        make_node('bc', {'b'}, {'c'}),

        make_node('_ab', {'_a'}, {'_b'}),
        make_node('_ac', {'_a'}, {'_c'}),
        make_node('_ay', {'_a'}, {'_y'}),

        make_node('+1', {'b', '_b'}, {'+1'}),
        make_node('+2', {'c', '_b', 'a'}, {'+2', 'a'}),
        make_node('+3', {'+1', '+2', '_y', '_c', 'c'}, {'+2', 'j', '+3', 'x'}),

        make_node('end', {'+1', '+2', '+3'}, {'+x'})
    ],
)

complex:
sol len diff
[<{s}->{a},{_a}>, <{a}->{b}>, <{b}->{c}>, <{_a}->{_b}>, <{_a}->{_c}>, <{_a}->{_y}>, <{b},{_b}->{+1}>, <{a},{c},{_b}->{a},{+2}>, <{c},{_c},{+1},{+2},{_y}->{j},{x},{+2},{+3}>, <{+1},{+2},{+3}->{+x}>]
failed to produce {'+x'}
no sol



In [3]:
todo = r.info
# s = todo.popleft()
print()

for s in todo:
    for a in s.plan:
        print(a)
    # print(s.have)
    print()
0

TypeError: 'State' object is not iterable

In [2]:
_test(
    'one',[
        make_node('ax', {'a'}, {'x'}),
        make_node('ay', {'a'}, {'y'}),
        make_node('jx', {'j'}, {'x'}),
    ],
    {'a'}, {'x'},
    [
        make_node('ax', {'a'}, {'x'}),
    ],
)

_test(
    'line',[
        make_node('ab', {'a'}, {'b'}),
        make_node('ay', {'a'}, {'y'}),
        make_node('bc', {'b'}, {'c'}),
        make_node('cx', {'c'}, {'x'}),
        make_node('jx', {'j'}, {'x'}),
    ],
    {'a'}, {'x'},
    [
        make_node('ab', {'a'}, {'b'}),
        make_node('bc', {'b'}, {'c'}),
        make_node('cx', {'c'}, {'x'}),
    ],
)

_test(
    'asymm diamond',[
        make_node('sss', {'s'}, {'ss'}),
        make_node('ssa', {'ss'}, {'a'}),
        make_node('ab', {'a'}, {'b'}),
        make_node('bc', {'b'}, {'c'}),
        make_node('ay', {'a'}, {'y'}),

        make_node('a_c', {'a'}, {'_c'}),
        # Node('bc.x', {'b', 'c'}, {'x'}),
        make_node('_cc.x', {'_c', 'c'}, {'x'}),
        make_node('jx', {'j'}, {'x'}),
    ],
    {'s'}, {'x'},
    [
        make_node('sss', {'s'}, {'ss'}),
        make_node('ssa', {'ss'}, {'a'}),
        make_node('ab', {'a'}, {'b'}),
        make_node('bc', {'b'}, {'c'}),
        make_node('a_c', {'a'}, {'_c'}),
        make_node('_cc.x', {'_c', 'c'}, {'x'}),
    ],
)


_test(
    'loop',[
        make_node('ca', {'c'}, {'a'}),
        make_node('sa', {'s'}, {'a'}),
        make_node('ab', {'a'}, {'b'}),
        make_node('ay', {'a'}, {'y'}),
        make_node('bc', {'b'}, {'c'}),
        make_node('cx', {'c'}, {'x'}),
        make_node('jx', {'j'}, {'x'}),
    ],
    {'s'}, {'x'},
    [
        make_node('sa', {'s'}, {'a'}),
        make_node('ab', {'a'}, {'b'}),
        make_node('bc', {'b'}, {'c'}),
        make_node('cx', {'c'}, {'x'}),
    ],
)

_test(
    'loop b',[
        make_node('sa', {'s'}, {'a'}),
        make_node('ca', {'c'}, {'a'}),
        make_node('ab', {'a'}, {'b'}),
        make_node('ay', {'a'}, {'y'}),
        make_node('bc', {'b'}, {'c'}),
        make_node('cx', {'c'}, {'x'}),
        make_node('jx', {'j'}, {'x'}),
    ],
    {'s'}, {'x'},
    [
        make_node('sa', {'s'}, {'a'}),
        make_node('ab', {'a'}, {'b'}),
        make_node('bc', {'b'}, {'c'}),
        make_node('cx', {'c'}, {'x'}),
    ],
)

_test(
    'loop vs fork',[
        make_node('sa', {'s'}, {'a'}),
        make_node('ab+', {'a'}, {'b', 'j', 'k'}),
        make_node('bk.c', {'b', 'k'}, {'c'}),
        make_node('c.x', {'c'}, {'x'}),
        make_node('ja', {'j'}, {'a'}),
    ],
    {'s'}, {'x'},
    [        
        make_node('sa', {'s'}, {'a'}),
        make_node('ab+', {'a'}, {'b', 'j', 'k'}),
        make_node('bk.c', {'b', 'k'}, {'c'}),
        make_node('c.x', {'c'}, {'x'}),
    ],
)

_test(
    'distributed',[
        make_node('ab', {'a'}, {'b'}),
        make_node('bc', {'b'}, {'c1'}),

        make_node('2', {'c1'}, {'c2'}),
        make_node('3', {'s1'}, {'c3'}),

        make_node('g', {'a', 'b', 'c1', 'c2', 'c3'}, {'g1', 'g2'}),

        make_node('*', {'c1', 'c2', 'c3', 's2', 'g1', 'g2'}, {'d'}),

        make_node('de', {'d'}, {'e'}),
        make_node('ex', {'e'}, {'x'})
    ],
    {'a', 's1', 's2'}, {'x'},
    [
        make_node('ab', {'a'}, {'b'}),
        make_node('bc', {'b'}, {'c1'}),

        make_node('2', {'c1'}, {'c2'}),
        make_node('3', {'s1'}, {'c3'}),
        
        make_node('g', {'a', 'b', 'c1', 'c2', 'c3'}, {'g1', 'g2'}),

        make_node('*', {'c1', 'c2', 'c3', 's2'}, {'d'}),

        make_node('de', {'d'}, {'e'}),
        make_node('ex', {'e'}, {'x'})
    ],
)

_test(
    'overlay',[
        make_node('s.abc', {'s'}, {'a', 'b', 'c'}),
        make_node('ab.jk', {'a', 'b'}, {'j', 'k'}),
        make_node('bc.ki', {'b', 'c'}, {'k', 'i'}),
        make_node('ca.ij', {'a', 'c'}, {'i', 'j'}),

        make_node('end', {'i', 'j', 'k'}, {'x'})
    ],
    {'s'}, {'x'},
    [
        make_node('s.abc', {'s'}, {'a', 'b', 'c'}),
        make_node('bc.ki', {'b', 'c'}, {'k', 'i'}),
        make_node('ca.ij', {'a', 'c'}, {'i', 'j'}),

        make_node('end', {'i', 'j', 'k'}, {'x'})
    ],
)

_test(
    'should pick shorter path',[
        make_node('s.ai', {'s'}, {'a', 'i'}),

        make_node('i.j', {'i'}, {'j'}),
        make_node('j.k', {'j'}, {'k'}),
        make_node('k.x', {'k'}, {'x'}),

        make_node('a.b', {'a'}, {'b'}),
        make_node('b.x', {'b'}, {'x'}),
    ],
    {"s"}, {"x"},
    [
        make_node('s.ai', {'s'}, {'a', 'i'}),
        make_node('a.b', {'a'}, {'b'}),
        make_node('b.x', {'b'}, {'x'}),
    ],
)

# {'+2'}
# {sa, ab, bc, _ab, _ay, _ac, +1, +3, end}
# {'j', '+1', '_y', 'b', 'x', 'c', '+x', '+3', 'a', '_a', '_c', '_b'}}
# _test(
#     'complex',[
#         make_node('sa', {'s'}, {'a', '_a'}),

#         make_node('ab', {'a'}, {'b'}),
#         make_node('ay', {'a'}, {'y'}),
#         make_node('bc', {'b'}, {'c'}),
#         make_node('ca', {'c'}, {'a'}),
#         make_node('cx', {'c'}, {'x'}),
#         make_node('jx', {'j'}, {'x'}),

#         make_node('_ab', {'_a'}, {'_b'}),
#         make_node('_ay', {'_a'}, {'_y'}),
#         make_node('_ac', {'_a'}, {'_c'}),
#         make_node('_bc.x', {'_b', '_c'}, {'_x'}),
#         make_node('_jx', {'_j'}, {'_x'}),

#         make_node('+1', {'b', '_b'}, {'+1'}),
#         make_node('+2', {'c', '_b', 'a'}, {'+2', 'a'}),
#         make_node('+3', {'+1', '+2', '_y', '_c', 'c'}, {'j', '+3', 'x'}),

#         make_node('end', {'+1', '+2', '+3'}, {'+x'})
#     ],
#     {'s'}, {'+x'},
#     [
#         make_node('sa', {'s'}, {'a', '_a'}),
#         make_node('ab', {'a'}, {'b'}),
#         make_node('bc', {'b'}, {'c'}),

#         make_node('_ab', {'_a'}, {'_b'}),
#         make_node('_ac', {'_a'}, {'_c'}),
#         make_node('_ay', {'_a'}, {'_y'}),

#         make_node('+1', {'b', '_b'}, {'+1'}),
#         make_node('+2', {'c', '_b', 'a'}, {'+2', 'a'}),
#         make_node('+3', {'+1', '+2', '_y', '_c', 'c'}, {'+2', 'j', '+3', 'x'}),

#         make_node('end', {'+1', '+2', '+3'}, {'+x'})
#     ],
# )

one:
one solved in 2 steps

line:
line solved in 5 steps

asymm diamond:
asymm diamond solved in 8 steps

loop:
sol len diff
[<{s}->{a}>, <{a}->{b}>, <{b}->{c}>, <{c}->{x}>]

Application(transform=<{s}->{a}>, used={<o200:s>}, produced={<p200:a>}, signature='7200-o200')
Application(transform=<{a}->{b}>, used={<p200:a>}, produced={<q200:b>}, signature='B200-p200')
Application(transform=<{b}->{c}>, used={<q200:b>}, produced={<s200:c>}, signature='J200-q200')
Application(transform=<{c}->{a}>, used={<s200:c>}, produced={<t200:a>}, signature='3200-s200')
Application(transform=<{a}->{b}>, used={<t200:a>}, produced={<u200:b>}, signature='B200-t200')
Application(transform=<{b}->{c}>, used={<u200:b>}, produced={<w200:c>}, signature='J200-u200')
Application(transform=<{c}->{a}>, used={<w200:c>}, produced={<x200:a>}, signature='3200-w200')
Application(transform=<{a}->{b}>, used={<x200:a>}, produced={<y200:b>}, signature='B200-x200')
Application(transform=<{b}->{c}>, used={<y200:b>}, produced={<+20

Result(solution=[Application(transform=<{s}->{a},{i}>, used={<B900:s>}, produced={<C900:a>, <D900:i>}, signature='Y800-B900'), Application(transform=<{i}->{j}>, used={<D900:i>}, produced={<E900:j>}, signature='d800-D900'), Application(transform=<{j}->{k}>, used={<E900:j>}, produced={<F900:k>}, signature='h800-E900'), Application(transform=<{k}->{x}>, used={<F900:k>}, produced={<G900:x>}, signature='l800-F900'), Application(transform=<{x}->>, used={<G900:x>}, produced=set(), signature='8900-G900')], message='', info=None, steps=5, success=True)